# The shallow water equations

The 1D linearized shallow water equations (SWE) are given by

$$\begin{aligned}
\frac{\partial u}{\partial t}+U\frac{\partial u}{\partial x}+g\frac{\partial h}{\partial x} &=0 \\
\frac{\partial h}{\partial t}+H\frac{\partial u}{\partial x}+U\frac{\partial h}{\partial x} &=0
\end{aligned}$$

where the unknown fields are $u$ (horizontal velocity, superposed to $U$), $h$ (water height, superposed to $H$), and the fixed parameters are $U$ (background horizontal velocity), $H$ (background water height) and $g$ (gravity constant).



## Preparations: load numpy and matplotlib libraries

As usual, first some auxiliary libraries and functions are loaded.

In [ ]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(phi,frames=None,labels=None,ylim=None):
    
    # create figure with axes
    fig,ax=plt.subplots()
    
    if len(phi.shape)==2:
        phi=phi.reshape([phi.shape[0],phi.shape[1],1])
    
    # number of experiments
    nExp=phi.shape[2]

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=phi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    
    # set y limits
    if ylim is None:
        ymin=np.min(phi[0,:,:])
        ymax=np.max(phi[0,:,:])
        ylim=[1.2*ymin-0.2*ymax,1.2*ymax-0.2*ymin]
    
    # labels and legend   
    showlegend=True
    if labels is None:
        showlegend=False
        labels=['exp %i'%jExp for jExp in range(nExp)]
        
    # create lines for numerical solution
    ll_n=[ax.plot(x,phi[frames[0],:,jExp],label=labels[jExp])[0] for jExp in range(nExp)]
    
    # set ylim
    ax.set_ylim(ylim)
    
    # add legend
    if showlegend:
        plt.legend()
    
    # add title
    tt=plt.title('')
    

    def animate(iframe):
        for jExp in range(nExp):
            ll_n[jExp].set_ydata(phi[iframe,:,jExp])
        tt.set_text('timestep %i/%i'%(iframe,nt))

    anim=animation.FuncAnimation(fig, animate, frames=frames,cache_frame_data=False,blit=True)
    display(anim)

## Solving the SWE

The 1D linearized shallow water equations can be solved with a leapfrog time scheme and centered finite differences:

$$\begin{aligned}
\frac{u^{n+1}_{j}-u^{n-1}_{j}}{2\Delta t}+U\frac{u^{n}_{j+1}-u^{n}_{j-1}}{2\Delta x}+g\frac{h^{n}_{j+1}-h^{n}_{j-1}}{2\Delta x} &=0 \\
\frac{h^{n+1}_{j}-h^{n-1}_{j}}{2\Delta t}+U\frac{h^{n}_{j+1}-h^{n}_{j-1}}{2\Delta x}+H\frac{u^{n}_{j+1}-u^{n}_{j-1}}{2\Delta x} &=0
\end{aligned}$$

where the superscripts $n-1$, $n$, $n+1$ denote the solution at the previous, current and next timestep, while the subscripts $j-1$, $j$, $j+1$ denote the gridpoint index.

From these equations, the next timestep's solution is found as

$$\begin{aligned}
u^{n+1}_{j}=u^{n-1}_{j}-\frac{\Delta t}{\Delta x}\left(U(u^{n}_{j+1}-u^{n}_{j-1})+g(h^{n}_{j+1}-h^{n}_{j-1})\right) \\
h^{n+1}_{j}=h^{n-1}_{j}-\frac{\Delta t}{\Delta x}\left(U(h^{n}_{j+1}-h^{n}_{j-1})+H(u^{n}_{j+1}-u^{n}_{j-1})\right)
\end{aligned}$$

During the first timestep, the forward scheme is used since there is no previous timestep yet.

In [ ]:
# solving with leapfrog+2nd order centered differences

# parameters
dt=0.1
nt=300

dx=1
nx=128

U=1
H=1
g=9.81

# courant number
mu=(np.abs(U)+np.sqrt(g*H))*dt/dx
print('Courant number : ',mu)

# grid
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# allocate solution
h=np.zeros((nt+1,nx))
u=np.zeros((nt+1,nx))

# initial state: gaussian bell for water height
h[0,:]=0.5*np.exp(-40*(x/(nx*dx)-0.5)**2)

# indices of points to the left and right
j=np.arange(nx)
jL=(j-1)%nx
jR=(j+1)%nx

# first timestep: forward scheme
jt=0
u[jt+1,:]=u[jt,:]-dt*(U*(u[jt,jR]-u[jt,jL])+g*(h[jt,jR]-h[jt,jL]))/(2*dx)
h[jt+1,:]=h[jt,:]-dt*(H*(u[jt,jR]-u[jt,jL])+U*(h[jt,jR]-h[jt,jL]))/(2*dx)

# later timesteps: leapfrog
for jt in range(1,nt):
    u[jt+1,:]=u[jt-1,:]-dt/dx*(U*(u[jt,jR]-u[jt,jL])+g*(h[jt,jR]-h[jt,jL]))
    h[jt+1,:]=h[jt-1,:]-dt/dx*(H*(u[jt,jR]-u[jt,jL])+U*(h[jt,jR]-h[jt,jL]))
    
# animate result (height) with about 30 frames
create_animation(h,frames=np.arange(0,nt+1,nt//30))
